In [2]:
import os
import tqdm
from PIL import Image

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import lightgbm as lgb

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

import torchvision
from torchvision import models
from torchvision import datasets
from torch.utils.data.dataloader import DataLoader
from torchvision.transforms import Compose
from torchvision.transforms import transforms
from torchvision.transforms import RandomCrop, RandomHorizontalFlip, Normalize

import matplotlib.pyplot as plt
from tensorboardX import SummaryWriter
from tools.csv_preprocessed_util import preprocess_pipeline

writer = SummaryWriter()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 1. 데이터 불러오기

## 1) CSV 데이터 최종 전처리

In [2]:
df = pd.read_csv("../../csv/data_regression_clean.csv")
df.info() # 1391개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1391 entries, 0 to 1390
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    1391 non-null   int64  
 1   id            1391 non-null   object 
 2   title         1391 non-null   object 
 3   detail        1391 non-null   object 
 4   condition     1391 non-null   object 
 5   is_completed  1391 non-null   bool   
 6   price         1391 non-null   float64
 7   location      1091 non-null   object 
 8   source        1391 non-null   object 
 9   model         1391 non-null   object 
 10  model_type    640 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(8)
memory usage: 110.2+ KB


In [3]:
config = {
    # 사용할 칼럼만
    "select_cols": ["id", "condition", "is_completed", "location", "model", "model_type", "price"],

    # 범주형 라벨 인코딩 (id는 제외)
    "label_encode": {"cols": ["condition", "is_completed", "location", "model", "model_type"]},

    # price는 타깃이므로 따로 스케일링/인코딩 안함
    "final_cols": ["id", "condition", "is_completed", "location", "model", "model_type", "price"]
}

# fit 모드 (train 데이터)
df_processed, artifacts = preprocess_pipeline(df, mode="fit", config=config)

# val/test에 적용할 때
# df_val_processed, _ = preprocess_pipeline(df_val, mode="transform", artifacts=artifacts, config=config)

In [4]:
artifacts['label_encoders']['condition'].classes_

array(['사용감 많음', '사용감 적음', '새 상품'], dtype=object)

## 2) 이미지 데이터 불러오기

In [5]:
def load_images_by_uuid(image_root_path, uuid_list):
    """
    주어진 UUID 리스트에 해당하는 이미지들을 불러와 딕셔너리로 반환하는 함수입니다.
    Args:
        image_root_path (str): 이미지가 저장된 루트 디렉토리 경로 (예: 'C:\Potenup\SecondHanded-Strollers-PredictedPrice\data\preprocessed\images')
        uuid_list (list): 이미지를 불러올 UUID 문자열 리스트.
    Returns:
        dict: {uuid: [Image1, Image2, ...]} 형식의 딕셔너리
    """
    image_data_dict = {}
    error_image_list = []
    count= 0
    for uuid in uuid_list:
        uuid_path = os.path.join(image_root_path, uuid)
        if not os.path.isdir(uuid_path):    
            error_image_list.append(uuid)
            print(f"경고: {uuid} 경로를 찾을 수 없습니다.")
            continue
        
        image_list = []
        for filename in os.listdir(uuid_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(uuid_path, filename)
                try:
                    img = Image.open(image_path).convert('RGB')
                    image_list.append(img)
                except Exception as e:
                    print(f"오류: {image_path} 이미지 파일을 불러올 수 없습니다. 오류: {e}")
        
        if image_list:
            image_data_dict[uuid] = image_list
        else:
            print(f"정보: {uuid_path} 폴더에 이미지가 없습니다.")
    
    return image_data_dict, error_image_list

target_uuids_list = df['id'].tolist()

image_base_path = '../../data/total_images'
uuid_images, error_image_list = load_images_by_uuid(image_base_path, target_uuids_list)
print(f"\n이미지들을 불러오는 중...")

uuid_images[target_uuids_list[0]]


이미지들을 불러오는 중...


[<PIL.Image.Image image mode=RGB size=480x480>,
 <PIL.Image.Image image mode=RGB size=480x480>,
 <PIL.Image.Image image mode=RGB size=480x480>,
 <PIL.Image.Image image mode=RGB size=480x480>,
 <PIL.Image.Image image mode=RGB size=480x480>,
 <PIL.Image.Image image mode=RGB size=480x480>,
 <PIL.Image.Image image mode=RGB size=480x480>]

In [6]:
len(target_uuids_list)

1391

In [7]:
len(error_image_list)

0

In [8]:
# 이미지를 변환및 크롭 함수 정의
def get_image_transforms(target_size=224):
    """
    이미지 변환 파이프라인 정의
    여러 이미지 크롭 및 리사이즈 방법을 정의
    Compose를 사용하여 파이프라인을 만듭니다.
    """
    def custom_crop_and_resize(img):
        """
        이미지의 비율에 따라 크롭 또는 패딩을 적용하여 1:1 비율로 만드는 내부 함수입니다.
        """
        width, height = img.size
        # 가로 세로 비율 차이가 크면 크롭을 적용합니다.
        if abs(width / height - 1.0) > 0.1:
            if width > height:
                img = transforms.CenterCrop((height, height))(img)
            else:
                img = transforms.CenterCrop((width, width))(img)
        # 비율 차이가 작으면 패딩을 적용하여 정사각형으로 만듭니다.
        else:
            max_side = max(width, height)
            new_img = Image.new('RGB', (max_side, max_side), (0, 0, 0))
            new_img.paste(img, ((max_side - width) // 2, (max_side - height) // 2))
            img = new_img
        
        return img.resize((target_size, target_size))

    return Compose([
        transforms.Lambda(lambda img: custom_crop_and_resize(img)),
        transforms.ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

# 2. 데이터 셋 만들기

In [9]:
# 데이터셋 및 데이터로더 준비
class CombinedDataset(torch.utils.data.Dataset):
    """
    이미지 데이터와 csv 데이터를 함께 처리하기 위한 필수적인 Dataset 클래스
    """
    def __init__(self, df, image_ids, target_id, train_ids, image_transform=None):
        self.df = df
        self.transform = image_transform
        self.image_ids = image_ids
        self.labels = df[target_id]

        self.tabular_data = self.df[train_ids]
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        uuid_path = os.path.join(image_base_path, image_id)
        filename = os.listdir(uuid_path)[0]
        image_path = os.path.join(uuid_path, filename)
        image = Image.open(image_path).convert('RGB')
        
        # 이미지 크롭 추가

        image_tensor = self.transform(image)
        tabular_tensor = torch.tensor(self.tabular_data.iloc[idx].values, dtype=torch.float32)
        tabular_tensor = tabular_tensor
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        
        return image_tensor, tabular_tensor, label

def prepare_data_and_loaders(df, target_id, train_ids, batch_size=32):
    """
    CSV 파일을 불러오고, 학습/검증 데이터셋으로 나눈 후, 데이터로더를 반환하는 함수입니다.
    """
    targets = df[target_id]
    df = df.dropna(subset=[target_id])
    
    # 'id' 컬럼을 리스트로 불러옵니다.
    id_list = df['id'].tolist()
    
    # 학습/검증 데이터로 분리
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
    
    train_df = train_df.reset_index(drop=True)
    val_df = val_df.reset_index(drop=True)

    image_transforms = get_image_transforms()
    
    train_dataset = CombinedDataset(df=train_df, image_ids = train_df['id'].tolist() , target_id = target_id, train_ids = train_ids, image_transform=image_transforms)
    val_dataset = CombinedDataset(df=val_df, image_ids = val_df['id'].tolist(), target_id= target_id,train_ids = train_ids, image_transform=image_transforms)
    
    train_dataLoader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataLoader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    tabular_data_size = len(train_dataset.tabular_data.columns)
    
    print(f"학습 데이터 크기: {len(train_df)}")
    print(f"검증 데이터 크기: {len(val_df)}")
    print(f"배치 사이즈: {batch_size}")
    print(f"전체 ID 개수: {len(id_list)}")
    
    return train_dataLoader, val_dataLoader, tabular_data_size, id_list

In [10]:
# 메인 실행 부분 (데이터 로더 호출)
train_loader, val_loader, tabular_size, id_list = prepare_data_and_loaders(df_processed, target_id = 'price', train_ids = ['is_completed', 'location', 'model', 'model_type', 'condition'], batch_size=32)

학습 데이터 크기: 1112
검증 데이터 크기: 279
배치 사이즈: 32
전체 ID 개수: 1391


In [ ]:
image_tensor, tabular_tensor, label = next(iter(train_loader))
image_tensor[0]
image_tensor.shape

torch.Size([32, 3, 224, 224])

# 3. 전이 학습

In [5]:
model = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 1)  # 회귀용
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# 4. 모델 학습

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import resnet34, ResNet34_Weights

class ResNet34TabularRegressor(nn.Module):
    def __init__(self, tab_in: int):
        super().__init__()
        m = resnet34(weights=ResNet34_Weights.DEFAULT)
        feat_dim = m.fc.in_features
        m.fc = nn.Identity()          # GAP features만 받기
        self.backbone = m

        # 탭 임베딩(탭 영향력 ↑)
        self.tab_mlp = nn.Sequential(
            nn.Linear(tab_in, 128), nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 256), nn.ReLU()
        )
        self.tab_scale = nn.Parameter(torch.tensor(2.0))  # 탭 가중치(학습가능)

        self.head = nn.Sequential(
            nn.Linear(feat_dim + 256, 256), 
            nn.ReLU(),
            nn.Linear(256, 64), nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, images, tab):
        f_img = self.backbone(images)             # (B, 512)
        f_tab = self.tab_mlp(tab)                 # (B, 256)
        fused = torch.cat([f_img, self.tab_scale * f_tab], dim=1)
        return self.head(fused)


# 5. 모델 평가

모델 평가 시작...


100%|██████████| 9/9 [00:01<00:00,  4.76it/s]

검증 데이터셋의 평균 손실(MSE): 87022445909.3333
검증 MAE:  243,515.88
검증 RMSE: 86,818,578,432.00
검증 R²:   -0.5369
모델 평가 완료!
